In [ ]:
import os

import numpy as np
import pandas as pd

evaluation_dir = os.path.join(os.getcwd(), '..', 'evaluation', 'signals')
metadata_file = os.path.join(evaluation_dir, 'metadata.csv')

metadata = pd.read_csv(metadata_file)

# Parse the text to a numpy array
metadata['signal'] = metadata['signal'].apply(eval).apply(np.array)

# Parse the parameter to a dictionary
# metadata['parameter'] = metadata['parameter'].apply(eval).apply(dict)

# Parse the execution_time as a duration
metadata['execution_time'] = pd.to_timedelta(metadata['execution_time'])

metadata.head()

In [ ]:
import respiratory_extraction.dataset as repository

dataset = repository.from_default()

In [ ]:
from respiratory_extraction.models.frequency_extraction import FrequencyExtractor

evaluation_results = []

for index, row in metadata.iterrows():
    subject, scenario = row['subject'], row['scenario']

    if not dataset.contains(subject, scenario):
        # print(f'Skip {subject}/{scenario}')
        continue

    gt_signal, gt_sampling_rate = dataset.get_ground_truth_rr_signal(subject, scenario)
    gt_extractor = FrequencyExtractor(
        gt_signal,
        gt_sampling_rate,
        lowpass=0.1,
        highpass=0.6,
    )
    gt_frequencies = {
        'fft': gt_extractor.frequency_from_fft(),
        'pc': gt_extractor.frequency_from_peaks(),
        'cp': gt_extractor.frequency_from_crossing_point(),
        'nfcp': gt_extractor.frequency_from_nfcp(),
    }

    frequency_extractor = FrequencyExtractor(
        row['signal'],
        row['sampling_rate'],
        lowpass=0.1,
        highpass=0.6,
    )
    predicted_frequencies = {
        'fft': frequency_extractor.frequency_from_fft(),
        'pc': frequency_extractor.frequency_from_peaks(),
        'cp': frequency_extractor.frequency_from_crossing_point(),
        'nfcp': frequency_extractor.frequency_from_nfcp(),
    }

    for method in gt_frequencies.keys():
        # The frequency is in Hz, we want to convert it to bpm
        predicted_hz = predicted_frequencies[method] * 60
        gt_hz = gt_frequencies[method] * 60

        evaluation_results.append({
            'subject': subject,
            'scenario': scenario,
            'strategy': row['method'],
            'method': method,
            'gt_frequency': predicted_hz,
            'predicted_frequency': gt_hz,
            'error': abs(gt_hz - predicted_hz),
        })

In [ ]:
evaluation_results = pd.DataFrame(evaluation_results)
evaluation_results

In [ ]:
# Plot the mean error for each method
import matplotlib.pyplot as plt

methods = evaluation_results['method'].unique()
mae = []
mse = []

for method in methods:
    method_results = evaluation_results[
        (evaluation_results['method'] == method) &
        (evaluation_results['strategy'] == 'pixel_intensity')
        ]

    mean_absolut_error = method_results['error'].mean()
    mae.append(mean_absolut_error)

    mean_squared_error = (method_results['error'] ** 2).mean()
    mse.append(mean_squared_error)

fig, ax = plt.subplots(1, 2, figsize=(15, 5))

ax[0].bar(methods, mae)
ax[0].set_title('Overall MAE')
ax[0].set_ylabel('Error (bpm)')
ax[0].set_xlabel('Method')

ax[1].bar(methods, mse)
ax[1].set_title('Overall MSE')
ax[1].set_ylabel('Error (bpm)')
ax[1].set_xlabel('Method')

In [ ]:
# Create a bar plot for each scenario that averages the error for each method
scenarios = evaluation_results['scenario'].unique()

_, ax = plt.subplots(len(scenarios), 1, figsize=(15, 15))

for i, scenario in enumerate(scenarios):
    scenario_results = evaluation_results[evaluation_results['scenario'] == scenario]

    mae = []
    mse = []

    for method in methods:
        method_results = scenario_results[scenario_results['method'] == method]
        mean_absolut_error = method_results['error'].mean()
        mae.append(mean_absolut_error)

        mean_squared_error = (method_results['error'] ** 2).mean()
        mse.append(mean_squared_error)

    ax[i].bar(methods, mae)
    ax[i].set_title(f'{scenario} - MAE')
    ax[i].set_ylabel('Error (bpm)')
    ax[i].set_xlabel('Method')